<a href="https://colab.research.google.com/github/HungYangChang/ECSE551/blob/master/Miniproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cd content/drive/My Drive/Colab/ECSE551Miniproject1/
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('./hepatitis.csv')

In [ ]:
print(data)

In [ ]:
# Analysize Data

# Find mean
print(data.mean())
# Find correlation of each one (to check how correct of i.i.d assumpation)
corr_data = data.corr()
plt.figure(figsize=(20, 20))
heatmap = sns.heatmap(corr_data, annot=True, linewidths=0.5, linecolor="white",vmin=0.3, cmap="YlGnBu")

In [ ]:
# divide data into Training, validation, and Test (Q: is validation necessary?)
# k-fold non-finished yet

# 80% for training
data_train = data.sample(frac = 0.8, random_state=1)
##.iloc to get row information but not include label
print (data_train.iloc[:, :-1])
x_train = np.array(data_train.iloc[:, :-1])
print (x_train)
y_train = np.array(data_train["ClassLabel"])
print (y_train)

## Select half of rest data for validation (?)
# data_temp = data.drop(data_train.index)
# data_validation = data_temp.sample(frac = 0.5, random_state=1)
# x_validation = np.array(data_validation.iloc[:, :-1])
# y_validation = np.array(data_validation["ClassLabel"])

## Select rest of data for testing
print (data_train.index)
data_test = data.drop(data_train.index)
x_test = np.array(data_test.iloc[:, :-1])
y_test = np.array(data_test["ClassLabel"])

## Show number of training and testing data
print("Number of Training data: ", len(x_train),"Testing data:",len(x_test))

In [ ]:
# linear classifier logistic regression


In [ ]:
# fit

In [ ]:
# predict

In [ ]:
# Accu-eval

In [ ]:
# k-fold

In [ ]:
# Changing learning rate, check if it's overfit